# Subjectivity in News Articles

## Group:
- Luca Babboni - luca.babboni2@studio.unibo.it
- Matteo Fasulo - matteo.fasulo@studio.unibo.it
- Luca Tedeschini - luca.tedeschini3@studio.unibo.it

## Description

This notebook addresses Task 1 proposed in [CheckThat Lab](https://checkthat.gitlab.io/clef2025/) of CLEF 2025. In this task, systems are challenged to distinguish whether a sentence from a news article expresses the subjective view of the author behind it or presents an objective view on the covered topic instead.

This is a binary classification tasks in which systems have to identify whether a text sequence (a sentence or a paragraph) is subjective (SUBJ) or objective (OBJ).

The task comprises three settings:

* Monolingual: train and test on data in a given language
* Multilingual: train and test on data comprising several languages
* Zero-shot: train on several languages and test on unseen languages

training data in five languages:
* Arabic
* Bulgarian
* English
* German
* Italian

The official evaluation is macro-averaged F1 between the two classes.

# Installing dependencies

This notebook uses quantized models, and some additional libraries are required. If you are running this notebook on either Colab or Kaggle, please run the cell below once, then run the whole notebook normally.



In [ ]:
%%capture
%pip install -U transformers[torch] bitsandbytes trl peft sacremoses ctranslate2 accelerate

In [ ]:
import os
os.kill(os.getpid(), 9)

## Importing libraries

In [ ]:
import os
import gc
from pathlib import Path

import csv

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.nn.functional as F

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from sentence_transformers import SentenceTransformer
from datasets import Dataset
from huggingface_hub import notebook_login
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding, 
    BitsAndBytesConfig,
    PreTrainedModel,
    DebertaV2Model, 
    DebertaV2Config, 
    pipeline
)
from transformers.trainer_utils import PredictionOutput
from transformers.models.deberta.modeling_deberta import ContextPooler

## Setting the device

In [ ]:
SEED = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tqdm.pandas() # display tqdm on pandas apply functions
print(f"Using device: {device}")

Using device: cuda


## Setting Library Seeds

This step is necessary to guarantee reproducibility.



In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

## Setting the Data Folder  

Please modify this path with your dataset's local path. The `data` folder should follow the official challenge structure:
```
data/
|---- arabic/
|--------- xxxx.tsv
|---- bulgarian/
|--------- xxxx.tsv
|---- english/
|--------- xxxx.tsv
|---- german/
|--------- xxxx.tsv
|---- italian/
|--------- xxxx.tsv
```

In [ ]:
data_folder = '/kaggle/input/clef2025-checkthat/data' # data

## Hugging face notebook login

To correctly download and use the hugging face models, a token key needs to be provided. Please refer to this [page](https://huggingface.co/docs/hub/security-tokens)

In [ ]:
notebook_login()

# Subjectivity Class  
This class is used throughout the whole notebook as a utility toolbox to avoid code redundancy. When a method of this class is called for the first time, its behavior will be explained.  

In [ ]:
class Subjectivity:
    """
    A class for handling multilingual subjectivity classification datasets.
    
    This class provides functionality to load, process, and prepare datasets for 
    subjective/objective text classification across multiple languages. It supports:
    - Loading and organizing datasets from multiple languages
    - Splitting data into train/dev/test sets
    - Analyzing label distributions
    - Loading pre-trained tokenizers and models
    - Computing class weights for imbalanced datasets
    
    Attributes:
        seed (int): Random seed for reproducibility
        device (str): Computation device ('cuda' or 'cpu')
        languages (list): List of available languages in the dataset
        dataset (pd.DataFrame): Combined dataset with all languages and splits
        train (pd.DataFrame): Training split of the dataset
        dev (pd.DataFrame): Development split of the dataset
        test (pd.DataFrame): Test split of the dataset
        all_data (dict): Nested dictionary organizing data by language and split
        tokenizer (AutoTokenizer, optional): Hugging Face tokenizer
        model (AutoModelForSequenceClassification, optional): Classification model
    """
    def __init__(self, data_folder: str = 'data', seed: int = 42, device: str = 'cuda'):
        """
        Initialize the Subjectivity class.
        
        Args:
            data_folder (str): Directory path containing the dataset files.
            seed (int): Random seed for reproducibility.
            device (str): Device to use for computations ('cuda' or 'cpu').
        """
        self.seed = seed
        self.device = device
        self.languages = [language for language in os.listdir(data_folder)]

        dataset = self.create_dataset(data_folder=data_folder)
        self.dataset = dataset
        
        train, dev, test = self.get_splits(dataset, print_shapes=True)
        self.train = train
        self.dev = dev
        self.test = test

        self.all_data = self.get_per_lang_dataset()
        

    def create_dataset(self, data_folder: str = 'data'):
        """
        Create a consolidated dataset from files in multiple languages.
        
        Args:
            data_folder (str): Directory path containing subdirectories for each language.
            
        Returns:
            pd.DataFrame: Combined dataset with columns for sentence_id, sentence, label, 
                          language, and split information.
        """
        dataset = pd.DataFrame(columns=['sentence_id','sentence','label','lang','split'])
        for language in os.listdir(data_folder):
            for filename in os.listdir(f"{data_folder}{os.sep}{language}"):
                if '.tsv' in filename:
                    abs_path = f"{data_folder}{os.sep}{language}{os.sep}{filename}"
                    df = pd.read_csv(abs_path, sep='\t', quoting=csv.QUOTE_NONE)
                    if 'solved_conflict' in df.columns:
                        df.drop(columns=['solved_conflict'], inplace=True)
                    df['lang'] = language
                    df['split'] = Path(filename).stem
                    dataset = pd.concat([dataset, df], axis=0)
        return dataset

    def get_splits(self, dataset: pd.DataFrame, print_shapes: bool = True):
        """
        Split the dataset into training, development, and test sets.
        
        Args:
            dataset (pd.DataFrame): The combined dataset to split.
            print_shapes (bool): Whether to print the shapes of the resulting splits.
            
        Returns:
            tuple: A tuple containing three pandas DataFrames (train, dev, test).
        """
        train = dataset[dataset['split'].str.contains('train')].copy()
        dev = dataset[(dataset['split'].str.contains('dev')) & ~(dataset['split'].str.contains('dev_test'))].copy()
        test = dataset[dataset['split'].str.contains('dev_test')].copy()

        # encode the target variable to int (0: obj; 1: subj)
        train.loc[:, 'label'] = train['label'].apply(lambda x: 0 if x == 'OBJ' else 1)
        dev.loc[:, 'label'] = dev['label'].apply(lambda x: 0 if x == 'OBJ' else 1)
        test.loc[:, 'label'] = test['label'].apply(lambda x: 0 if x == 'OBJ' else 1)

        # cast to int
        train['label'] = train['label'].astype(int)
        dev['label'] = dev['label'].astype(int)
        test['label'] = test['label'].astype(int)

        if print_shapes:
            print(f"Train: {train.shape}")
            print(f"Dev: {dev.shape}")
            print(f"Test: {test.shape}")
            
        return train, dev, test

    def get_per_lang_dataset(self):
        """
        Organize the dataset by language and split (train, dev, test).
        
        Returns:
            dict: A nested dictionary with languages as the outer keys and
                  split names ('train', 'dev', 'test') as inner keys.
                  For example:
                  {
                      'english': {
                          'train': pd.DataFrame,
                          'dev': pd.DataFrame,
                          'test': pd.DataFrame
                      },
                      ...
                  }
        """
        dataset_dict = {}
        for language in self.languages:
            dataset_dict[language] = {}
            # get the train data
            dataset_dict[language]['train'] = self.train[self.train['lang']==language].copy()
            # get the dev data
            dataset_dict[language]['dev'] = self.dev[self.dev['lang']==language].copy()
            # get the test data
            dataset_dict[language]['test'] = self.test[self.test['lang']==language].copy()
        return dataset_dict

    def print_label_distrib(self, dataset: pd.DataFrame):
        """
        Print the normalized distribution of labels in the dataset.
        
        Args:
            dataset (pd.DataFrame): The dataset containing a 'label' column.
            
        Returns:
            None: Prints the percentage distribution of each label.
        """
        print(dataset['label'].value_counts(normalize=True))

    def get_tokenizer(self, model_card: str = "microsoft/mdeberta-v3-base"):
        """
        Load a tokenizer from the Hugging Face model hub.
        
        Args:
            model_card (str): Identifier for the pre-trained tokenizer to load.
            
        Returns:
            AutoTokenizer: The loaded tokenizer.
        """
        tokenizer = AutoTokenizer.from_pretrained(model_card)
        self.tokenizer = tokenizer
        return tokenizer

    def get_model(self, model_card: str = "microsoft/mdeberta-v3-base", *args, **kwargs):
        """
        Load a pre-trained model from the Hugging Face model hub.
        
        Args:
            model_card (str): Identifier for the pre-trained model to load.
            *args: Variable length argument list to pass to the model constructor.
            **kwargs: Arbitrary keyword arguments to pass to the model constructor.
            
        Returns:
            AutoModelForSequenceClassification: The loaded model.
        """
        model = AutoModelForSequenceClassification.from_pretrained(model_card, *args, **kwargs)
        self.model = model
        return model

    def get_class_weights(self, dataset: pd.DataFrame):
        """
        Compute class weights for imbalanced datasets.
        
        Args:
            dataset (pd.DataFrame): Dataset containing a 'label' column.
            
        Returns:
            numpy.ndarray: Array of class weights where the index corresponds to the class label.
        """
        class_weights = compute_class_weight('balanced', classes=np.unique(dataset['label']), y=dataset['label'])
        return class_weights

## Creating Our Detector Object  

The `__init__()` method of the `Subjectivity` class will load the dataset, set the device and the seeds, and automatically create the `test`, `dev`, and `train` splits from the datasets. It will also convert the `SUBJ` and `OBJ` labels to their corresponding numerical versions, so they are ready to be fed into a model.  


In [ ]:
detector = Subjectivity(data_folder=data_folder, seed=SEED, device=device)

Train: (6418, 5)
Dev: (2401, 5)
Test: (2332, 5)


In [ ]:
detector.print_label_distrib(detector.train)
detector.print_label_distrib(detector.dev)
detector.print_label_distrib(detector.test)

label
0    0.631349
1    0.368651
Name: proportion, dtype: float64
label
0    0.612245
1    0.387755
Name: proportion, dtype: float64
label
0    0.657376
1    0.342624
Name: proportion, dtype: float64


In [ ]:
detector.all_data['german']['train']['sentence'].str.len().describe()

count    800.000000
mean     126.296250
std       67.334117
min       31.000000
25%       80.000000
50%      112.500000
75%      161.000000
max      625.000000
Name: sentence, dtype: float64

Here we create the `results` dictionary and the `predictions_dict` dictionary. They will be used to store all the model's output

In [ ]:
results = {}
predictions_dict = {}

# Custom Trainer  

This class extends the `Trainer` provided by Hugging Face. Since we needed to tweak some details in the training process, we opted to override some `Trainer` functions with custom ones.  


In [ ]:
class CustomTrainer(Trainer):
    """
    Custom Trainer class extending Hugging Face's Trainer with additional functionality:
    - Support for class weights to handle imbalanced datasets
    - Custom loss computation with weighted cross-entropy
    - Threshold optimization for binary classification
    - Custom prediction with threshold application
    """
    
    def __init__(self, *args, class_weights=None, weights_dtype=torch.float32, **kwargs):
        """
        Initialize the CustomTrainer.
        
        Args:
            class_weights (array-like, optional): Weights for each class to handle class imbalance.
            weights_dtype (torch.dtype): Data type for the class weights tensor.
            *args, **kwargs: Arguments passed to the parent Trainer class.
        """
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=weights_dtype).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Compute the training loss with optional class weighting.
        
        Args:
            model: The model to train
            inputs: The inputs and targets of the model
            return_outputs (bool): Whether to return the outputs along with the loss
            num_items_in_batch: Not used but kept for compatibility
            
        Returns:
            torch.Tensor or tuple: Loss value alone or with model outputs
        """
        # Extract labels
        labels = inputs.get("labels")

        # Forward pass
        outputs = model(**inputs)

        # Extract logits 
        logits = outputs.get('logits')

        # Compute loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

    def compute_best_threshold(self, dataset, ignore_keys=None, metric_key_prefix="test"):
        """
        Find the optimal classification threshold that maximizes macro F1 score.
        
        Args:
            dataset: The dataset to use for threshold optimization
            ignore_keys (list, optional): Keys to ignore in the model outputs
            metric_key_prefix (str): Prefix for metric keys in the output
            
        Returns:
            float: The optimal threshold value
        """
        # Get raw predictions from parent class
        output = super().predict(dataset, ignore_keys, metric_key_prefix)

        # Convert logits to probabilities using softmax (for binary classification)
        logits = output.predictions
        logits_tensor = torch.tensor(logits)
        probabilities = torch.softmax(logits_tensor, dim=-1).numpy()

        # Calculate optimal threshold
        labels = output.label_ids
        thresholds = np.linspace(0.1, 0.9, 100) 

        best_threshold = 0.5  # Default threshold
        best_f1 = 0

        for threshold in thresholds:
            predictions = (probabilities[:, 1] >= threshold).astype(int)
            _, _, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro", zero_division=0)
            
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold

        # Return the best threshold found
        return best_threshold
        
    def predict(self, dataset, threshold: float = 0.5, ignore_keys=None, metric_key_prefix="test"):
        """
        Generate predictions with a custom threshold for binary classification.
        
        Args:
            dataset: The dataset to generate predictions for
            threshold (float): The classification threshold (default: 0.5)
            ignore_keys (list, optional): Keys to ignore in model outputs
            metric_key_prefix (str): Prefix for metric keys in the output
            
        Returns:
            PredictionOutput: Object containing predictions, labels, and metrics
        """
        # Get raw predictions from parent class
        output = super().predict(dataset, ignore_keys, metric_key_prefix)
        
        # Convert logits to probabilities using softmax (for binary classification)
        logits = output.predictions
        logits_tensor = torch.tensor(logits)
        probabilities = torch.softmax(logits_tensor, dim=-1).numpy()
        
        final_predictions = (probabilities[:, 1] >= threshold).astype(int)

        # Update predictions in the output object
        return PredictionOutput(
            predictions=final_predictions,
            label_ids=output.label_ids,
            metrics=output.metrics
        )

## Text Tokenizer

In [ ]:
def tokenize_text(texts):
    """
    Tokenize text data using the current tokenizer.
    
    Args:
        texts (dict): Dictionary containing text data with a 'sentence' field
        
    Returns:
        dict: Dictionary with tokenized text features including input_ids, 
              attention_mask, and potentially token_type_ids
    """
    return tokenizer(texts['sentence'], padding=True, truncation=True, max_length=256, return_tensors='pt')

## Evaluation Metrics

In [ ]:
def evaluate_metrics(eval_pred):
    """
    Calculate evaluation metrics for subjectivity classification models.
    
    This function computes various performance metrics for classification results:
    - Accuracy: Overall correctness of predictions
    - Macro-averaged precision, recall, and F1: Averages across both classes with equal weight
    - Class-specific metrics: Precision, recall, and F1 specifically for the subjective class
    
    Args:
        eval_pred (tuple): Tuple containing (predictions, labels) where:
            - predictions: Raw model outputs/logits with shape (n_samples, n_classes)
            - labels: Ground truth labels with shape (n_samples,)
            
    Returns:
        dict: Dictionary containing the following metrics:
            - macro_F1: Macro-averaged F1 score across all classes
            - macro_P: Macro-averaged precision across all classes
            - macro_R: Macro-averaged recall across all classes
            - SUBJ_F1: F1 score for the subjective class (label 1)
            - SUBJ_P: Precision for the subjective class
            - SUBJ_R: Recall for the subjective class
            - accuracy: Overall accuracy
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    m_prec, m_rec, m_f1, m_s = precision_recall_fscore_support(labels, predictions, average="macro",
                                                                zero_division=0)
    p_prec, p_rec, p_f1, p_s = precision_recall_fscore_support(labels, predictions, labels=[1],
                                                                zero_division=0)

    return {
        'macro_F1': m_f1,
        'macro_P': m_prec,
        'macro_R': m_rec,
        'SUBJ_F1': p_f1[0],
        'SUBJ_P': p_prec[0],
        'SUBJ_R': p_rec[0],
        'accuracy': acc
    }

## Saving Predictions

In [ ]:
def save_predictions(test_data, predictions, filename: str, save_dir: str = 'results'):
    """
    Save model predictions to a TSV file with sentence IDs and predicted labels.
    
    Args:
        test_data: Dataset containing the 'sentence_id' field to match with predictions
        predictions: Array of binary predictions (0 for OBJ, 1 for SUBJ)
        filename: Name of the output file (should end with .tsv)
        save_dir: Directory to save the predictions file (default: 'results')
        
    Returns:
        str: Full path to the saved predictions file
    """
    os.makedirs(save_dir, exist_ok=True)
    pred_df = pd.DataFrame()
    pred_df['sentence_id'] = test_data['sentence_id']
    pred_df['label'] = predictions
    pred_df['label'] = pred_df['label'].apply(lambda x: 'OBJ' if x == 0 else 'SUBJ')

    predictions_filepath = os.path.join(save_dir, filename)
    pred_df.to_csv(predictions_filepath, index=False, sep='\t')

    print(f"Saved predictions into file:", predictions_filepath)
    return predictions_filepath

# Custom Model

In [ ]:
class CustomModel(PreTrainedModel):
    config_class = DebertaV2Config

    def __init__(self, config, sentiment_dim=3, num_labels=2, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.deberta = DebertaV2Model(config)
        self.pooler = ContextPooler(config)
        output_dim = self.pooler.output_dim
        self.dropout = nn.Dropout(0.1)

        self.classifier = nn.Linear(output_dim + sentiment_dim, num_labels)

    def forward(self, input_ids, positive, neutral, negative, attention_mask=None, labels=None):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)

        encoder_layer = outputs[0]
        pooled_output = self.pooler(encoder_layer)
        
        # Sentiment features as a single tensor
        sentiment_features = torch.stack((positive, neutral, negative), dim=1)  # Shape: (batch_size, 3)
        
        # Combine CLS embedding with sentiment features
        combined_features = torch.cat((pooled_output, sentiment_features), dim=1)
        
        # Classification head
        logits = self.classifier(self.dropout(combined_features))
        
        return {'logits': logits}

## Sentiment Pipeline for Data Augmentation

In [ ]:
pipe = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment", tokenizer="cardiffnlp/twitter-xlm-roberta-base-sentiment", top_k=None)

In [ ]:
def extract_sentiment(text):
    sentiments = pipe(text)[0]
    return {k:v for k,v in [(list(sentiment.values())[0], list(sentiment.values())[1]) for sentiment in sentiments]}

# Zero shot inference

## Utility function

This function can be then integrated in the Subjectivity class

In [11]:
def zero_shot_prepare_data(train_languages : list,
                           train : pd.DataFrame,
                           dev : pd.DataFrame,
                           test : pd.DataFrame):

    
    train_set = train[train["lang"].isin(train_languages)].copy()
    dev_set = dev[~dev["lang"].isin(train_languages)].copy()
    test_set = test[~test["lang"].isin(train_languages)].copy()

    return train_set, dev_set, test_set

In [12]:
def generate_name(names : tuple):
    generated_name = ""
    for x in names:
        generated_name = generated_name + x.capitalize()[:2]
    return generated_name

## Custom loop to test triplets

In [ ]:
model_card = "microsoft/mdeberta-v3-base"
tokenizer = detector.get_tokenizer(model_card=model_card)

epochs = 6
batch_size = 16
lr = 1e-5
weight_decay = 0.0
label_smoothing = 0.0

training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    #warmup_ratio=0.5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)

for i, group in enumerate(itertools.combinations(detector.languages, 3)):
    print(f"TESTING {i} - {group}")
    with torch.no_grad():
        torch.cuda.empty_cache()
    if 'model' in locals() or 'model' in globals():
        del model
        print("Model deleted!")
    gc.collect()
    
    group_name = generate_name(group)

    zs_train, zs_dev, zs_test = zero_shot_prepare_data(group, detector.train, detector.dev, detector.test)

    language = group_name+'-NoSentiment'


    
    train_data = Dataset.from_pandas(zs_train)
    dev_data = Dataset.from_pandas(zs_dev)
    test_data = Dataset.from_pandas(zs_test)
    
    train_data = train_data.map(tokenize_text, batched=True)
    dev_data = dev_data.map(tokenize_text, batched=True)
    test_data = test_data.map(tokenize_text, batched=True)
    
    class_weights = detector.get_class_weights(zs_train)

    model = detector.get_model(
        model_card=model_card, 
        num_labels=2, 
        id2label={0: 'OBJ', 1: 'SUBJ'}, 
        label2id={'OBJ': 0, 'SUBJ': 1},
        output_attentions = False,
        output_hidden_states = False
    )
    
    collator_fn = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=dev_data,
        data_collator=collator_fn,
        compute_metrics=evaluate_metrics,
        class_weights=class_weights,
    )

    trainer.train()

    best_thr = trainer.compute_best_threshold(dataset=dev_data)
    pred_info = trainer.predict(dataset=test_data, threshold=best_thr)
    predictions, labels = pred_info.predictions, pred_info.label_ids
    
    acc = accuracy_score(labels, predictions)
    m_prec, m_rec, m_f1, m_s = precision_recall_fscore_support(labels, predictions, average="macro",
                                                                zero_division=0)
    p_prec, p_rec, p_f1, p_s = precision_recall_fscore_support(labels, predictions, labels=[1],
                                                                zero_division=0)
    stats = {
            'macro_F1': m_f1,
            'macro_P': m_prec,
            'macro_R': m_rec,
            'SUBJ_F1': p_f1[0],
            'SUBJ_P': p_prec[0],
            'SUBJ_R': p_rec[0],
            'accuracy': acc
        }
    
    results[language] = stats

    
    with torch.no_grad():
        torch.cuda.empty_cache()
    if 'model' in locals() or 'model' in globals():
        del model
        print("Model deleted!")
    gc.collect()

    language = group_name+'-Sentiment'
    
    model = CustomModel(
        model_name=model_card, 
        num_labels=2, 
        sentiment_dim=3
    )



    zs_train[['positive', 'neutral', 'negative']] = zs_train.progress_apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')
    zs_dev[['positive', 'neutral', 'negative']] = zs_dev.progress_apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')
    zs_test[['positive', 'neutral', 'negative']] = zs_test.progress_apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')

    train_data = Dataset.from_pandas(zs_train)
    dev_data = Dataset.from_pandas(zs_dev)
    test_data = Dataset.from_pandas(zs_test)

    train_data = train_data.map(tokenize_text, batched=True)
    dev_data = dev_data.map(tokenize_text, batched=True)
    test_data = test_data.map(tokenize_text, batched=True)

    collator_fn = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = CustomTrainer(
        model = model,
        args = training_args,
        train_dataset = train_data,
        eval_dataset = dev_data,
        data_collator = collator_fn,
        compute_metrics = evaluate_metrics,
        class_weights=class_weights,
    )

    trainer.train()

    best_thr = trainer.compute_best_threshold(dataset=dev_data)
    pred_info = trainer.predict(dataset=test_data, threshold=best_thr)
    predictions, labels = pred_info.predictions, pred_info.label_ids
    
    acc = accuracy_score(labels, predictions)
    m_prec, m_rec, m_f1, m_s = precision_recall_fscore_support(labels, predictions, average="macro",
                                                                zero_division=0)
    p_prec, p_rec, p_f1, p_s = precision_recall_fscore_support(labels, predictions, labels=[1],
                                                                zero_division=0)
    stats = {
            'macro_F1': m_f1,
            'macro_P': m_prec,
            'macro_R': m_rec,
            'SUBJ_F1': p_f1[0],
            'SUBJ_P': p_prec[0],
            'SUBJ_R': p_rec[0],
            'accuracy': acc
        }
    
    results[language] = stats
    
    

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


TESTING 0 - ('english', 'italian')
Model deleted!


Map:   0%|          | 0/2443 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

Map:   0%|          | 0/1335 [00:00<?, ? examples/s]

...

100%|██████████| 1335/1335 [00:11<00:00, 119.28it/s]


Map:   0%|          | 0/2443 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

Map:   0%|          | 0/1335 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Macro F1,Macro P,Macro R,Subj F1,Subj P,Subj R,Accuracy
1,No log,0.725917,0.612220,0.629209,0.631277,0.603379,0.514403,0.729572,0.612421
2,No log,0.972189,0.634223,0.718251,0.640070,0.486129,0.757202,0.357977,0.694182
3,No log,0.955200,0.685101,0.702052,0.681131,0.596721,0.680798,0.531128,0.709906
4,0.504100,1.252779,0.653370,0.706423,0.653422,0.527744,0.720539,0.416342,0.698899
5,0.504100,1.394758,0.648759,0.707636,0.649911,0.518148,0.726316,0.402724,0.697327
6,0.504100,1.417408,0.653806,0.701681,0.653355,0.531060,0.710098,0.424125,0.697327


In [27]:
pd.DataFrame(results).T.sort_values(by='macro_F1', ascending=False).round(4)

,macro_F1,macro_P,macro_R,SUBJ_F1,SUBJ_P,SUBJ_R,accuracy
EnIt-NoSentiment,0.6147,0.6219,0.6135,0.5166,0.5661,0.4750,0.6397
EnIt-Sentiment,0.6022,0.6074,0.6012,0.5045,0.5451,0.4695,0.6262


In [ ]:
dataframe = pd.DataFrame(results)

In [ ]:
dataframe.to_csv("results.csv")

# Zero shot without Arabic language

In [129]:
with torch.no_grad():
    torch.cuda.empty_cache()

if 'model' in locals() or 'model' in globals():
    del model
    print("Model deleted!")

gc.collect()

Model deleted!


10705

In [131]:
language = 'english'

epochs = 6
batch_size = 16
lr = 1e-5
weight_decay = 0.0

In [132]:
model_card = "microsoft/mdeberta-v3-base"

tokenizer = detector.get_tokenizer(model_card=model_card)

# Load the config
config = DebertaV2Config.from_pretrained(
    model_card,
    num_labels=2,
    id2label={0: 'OBJ', 1: 'SUBJ'},
    label2id={'OBJ': 0, 'SUBJ': 1},
    output_attentions=False,
    output_hidden_states=False
)

# Initialize the custom model
model = CustomModel(config=config, sentiment_dim=3, num_labels=2)

# Load pretrained weights from the original DeBERTa model
model.deberta = DebertaV2Model.from_pretrained(model_card, config=config)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
detector.all_data[language]['train'][['positive', 'neutral', 'negative']] = detector.all_data[language]['train'].progress_apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')
detector.all_data[language]['dev'][['positive', 'neutral', 'negative']] = detector.all_data[language]['dev'].progress_apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')
detector.all_data[language]['test'][['positive', 'neutral', 'negative']] = detector.all_data[language]['test'].progress_apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')

Device set to use cuda:0
100%|██████████| 484/484 [00:04<00:00, 120.10it/s]


In [134]:
train_data = Dataset.from_pandas(detector.all_data[language]['train'])
dev_data = Dataset.from_pandas(detector.all_data[language]['dev'])
test_data = Dataset.from_pandas(detector.all_data[language]['test'])

train_data = train_data.map(tokenize_text, batched=True)
dev_data = dev_data.map(tokenize_text, batched=True)
test_data = test_data.map(tokenize_text, batched=True)

collator_fn = DataCollatorWithPadding(tokenizer=tokenizer)

class_weights = detector.get_class_weights(detector.all_data[language]['train'])

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

In [135]:
# Define training args
training_args = TrainingArguments(
    output_dir=f"mdeberta-v3-base-subjectivity-sentiment-{language}",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    #warmup_ratio=0.5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)

In [136]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = dev_data,
    data_collator = collator_fn,
    compute_metrics = evaluate_metrics,
    class_weights=class_weights,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro P,Macro R,Subj F1,Subj P,Subj R,Accuracy
1,No log,0.676271,0.638498,0.650187,0.644707,0.610329,0.698925,0.541667,0.640693
2,No log,0.492784,0.766216,0.768179,0.767736,0.764192,0.802752,0.729167,0.766234
3,No log,0.512452,0.767482,0.779157,0.772016,0.752887,0.844560,0.679167,0.768398
4,No log,0.506866,0.792145,0.795240,0.794088,0.788546,0.836449,0.745833,0.792208


In [ ]:
# Decision threshold calibration on dev set
best_thr = trainer.compute_best_threshold(dataset=dev_data)
# Predictions on dev set (with best threshold on dev set)
pred_info = trainer.predict(dataset=dev_data, threshold=best_thr)

predictions, labels = pred_info.predictions, pred_info.label_ids

# Save dev set predictions
save_predictions(dev_data, predictions, filename = f"dev_{language}_sentiment_predicted.tsv")

# Predictions on test set (with best threshold on dev set)
pred_info = trainer.predict(dataset=test_data, threshold=best_thr)

predictions, labels = pred_info.predictions, pred_info.label_ids

# Save test set predictions
save_predictions(test_data, predictions, filename = f"test_{language}_sentiment_predicted.tsv")

acc = accuracy_score(labels, predictions)
m_prec, m_rec, m_f1, m_s = precision_recall_fscore_support(labels, predictions, average="macro",
                                                            zero_division=0)
p_prec, p_rec, p_f1, p_s = precision_recall_fscore_support(labels, predictions, labels=[1],
                                                            zero_division=0)
stats = {
        'macro_F1': m_f1,
        'macro_P': m_prec,
        'macro_R': m_rec,
        'SUBJ_F1': p_f1[0],
        'SUBJ_P': p_prec[0],
        'SUBJ_R': p_rec[0],
        'accuracy': acc
    }

print(stats)
results[f"{language}-sentiment-thr"] = stats

cm = confusion_matrix(labels, predictions, normalize='all')
ConfusionMatrixDisplay(cm, display_labels=['OBJ', 'SUBJ']).plot()
plt.title(f"Confusion Matrix ({language})")
plt.show()